# Part 2: Cleanup of the cc-2003-2020.csv file 

1. Create a FIPS code based on the state and county integers. 
2. Expand the data from yearly to estimated monthly numbers

## 1. Create the FIPS code

In [12]:
import pandas as pd

df = pd.read_csv("clean/cc-2003-2020.csv")
df.head()

df['FIPS'] = df['STATE'].astype(str).str.pad(2,fillchar='0') + df['COUNTY'].astype(str).str.pad(3,fillchar='0')
df[['STATE','COUNTY', 'FIPS', ]].head()

,STATE,COUNTY,FIPS
0,39,1,39001
1,39,1,39001
2,39,1,39001
3,39,1,39001
4,39,1,39001


In [13]:
df[['STATE','COUNTY', 'FIPS']].tail()

,STATE,COUNTY,FIPS
396013,56,45,56045
396014,56,45,56045
396015,56,45,56045
396016,56,45,56045
396017,56,45,56045


In [16]:
df.to_csv("clean/cc-2003-2020.csv", index=False)

## 2. Linear Interpolation from yearly to monthly data [incomplete]

- Right now the data is in a yearly format, but we want it in a monthly estimated number for each of the 12 months in each year. 
- Each year estimate represents the population estimate for July of that year, so, 2006 is really July 2006 - June 2007.
- What we need to do is expand the data so that there are 12 observations per year-fips-agegrp and do a linear interpolation between the beginning and end of that data. so, in the example above, we see for trousdale county for 15-19 year olds in July 2006, there were 212 total females and in July 2007 there were 229 total 15-19 year olds. 

We need to expand the data (you can do this using the expand command if you want to keep it in long form, or reshape to wide form and generate new variables) and say for each month i since month 1, where month 1=July, the value for each of the columns (tot_female, nhwa_female nhba_female nhia_female nhaa_female nhna_female nhtom_fem and h_fem)= [population for month 1 + (((population for month 12-population for month 1)/12)*i)]. In this case 229-212=17, 17/12=1.42. So July 2006=212, August 2006=213.42, September 2006=2014.84, and so forth until you get to July 2007, which should equal 229.

In [8]:
df = df[0:1000]

display(df)


# get the estimate for the previous and next year 
for col in ['TOT_FEMALE']:
    df[col+ '_next_year'] = 0
    df[col+ '_prev_year'] = 0

    for index, row in df.iterrows():
        year = row['YEAR']
        agegrp = row['AGEGRP']
        fips = row['FIPS']


        # get next year's values 
        next_year_row = df.loc[(df['YEAR'] == year +1) & (df['AGEGRP'] == agegrp) & (df['FIPS'] == fips)][col]
        
        if len(next_year_row) ==0: 
            print(year, agegrp, fips, 'no next year')
        elif year!=2020:
            next_year_value = next_year_row.iloc[0]
            df.at[index,col+ '_next_year']  = next_year_value

        # get prev year's values 
        prev_year_row = df.loc[(df['YEAR'] == year -1) & (df['AGEGRP'] == agegrp) & (df['FIPS'] == fips)][col]
        
        if len(prev_year_row) ==0: 
            print(year, agegrp, fips, 'no prev year')
        elif year!= 2003:
            prev_year_value = prev_year_row.iloc[0]
            df.at[index,col+ '_prev_year']  = prev_year_row

df[df['AGEGRP'] == 4][['AGEGRP', 'FIPS', 'YEAR', 'TOT_FEMALE', 'TOT_FEMALE_next_year', 'TOT_FEMALE_prev_year']]

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_FEMALE,WA_FEMALE,BA_FEMALE,...,HIA_FEMALE,HAA_FEMALE,HNA_FEMALE,HTOM_FEMALE,HWAC_FEMALE,HBAC_FEMALE,HIAC_FEMALE,HAAC_FEMALE,HNAC_FEMALE,FIPS
0,50,39,1,Ohio,Adams County,2003,4,931,905,6,...,1,0,0,0,12.0,0.0,1.0,0.0,0.0,39001
1,50,39,1,Ohio,Adams County,2003,5,883,863,7,...,1,0,0,0,6.0,0.0,1.0,0.0,0.0,39001
2,50,39,1,Ohio,Adams County,2003,6,868,852,2,...,0,0,0,1,5.0,0.0,1.0,0.0,0.0,39001
3,50,39,1,Ohio,Adams County,2003,7,920,904,2,...,0,0,0,0,6.0,0.0,0.0,0.0,0.0,39001
4,50,39,1,Ohio,Adams County,2003,8,987,963,1,...,1,0,0,0,6.0,0.0,1.0,0.0,0.0,39001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,50,39,41,Ohio,Delaware County,2005,5,3358,3064,139,...,0,1,0,1,58.0,2.0,0.0,2.0,0.0,39041
996,50,39,41,Ohio,Delaware County,2005,6,4265,3806,178,...,0,1,0,1,90.0,5.0,1.0,1.0,0.0,39041
997,50,39,41,Ohio,Delaware County,2005,7,6350,5646,231,...,0,0,0,2,110.0,5.0,1.0,0.0,0.0,39041
998,50,39,41,Ohio,Delaware County,2005,8,6735,6110,234,...,0,1,0,0,100.0,4.0,0.0,1.0,0.0,39041


2003 4 39001 no prev year
2003 5 39001 no prev year
2003 6 39001 no prev year
2003 7 39001 no prev year
2003 8 39001 no prev year
2003 9 39001 no prev year
2003 10 39001 no prev year
2009 4 39001 no next year
2009 5 39001 no next year
2009 6 39001 no next year
2009 7 39001 no next year
2009 8 39001 no next year
2009 9 39001 no next year
2009 10 39001 no next year
2003 4 39003 no prev year
2003 5 39003 no prev year
2003 6 39003 no prev year
2003 7 39003 no prev year
2003 8 39003 no prev year
2003 9 39003 no prev year
2003 10 39003 no prev year
2009 4 39003 no next year
2009 5 39003 no next year
2009 6 39003 no next year
2009 7 39003 no next year
2009 8 39003 no next year
2009 9 39003 no next year
2009 10 39003 no next year
2003 4 39005 no prev year
2003 5 39005 no prev year
2003 6 39005 no prev year
2003 7 39005 no prev year
2003 8 39005 no prev year
2003 9 39005 no prev year
2003 10 39005 no prev year
2009 4 39005 no next year
2009 5 39005 no next year
2009 6 39005 no next year
2009 7 

/var/folders/4m/b6c49kjx5_526vwl3431rgcc0000gn/T/ipykernel_41867/104294003.py:33: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  df.at[index,col+ '_prev_year']  = prev_year_row


2003 4 39015 no prev year
2003 5 39015 no prev year
2003 6 39015 no prev year
2003 7 39015 no prev year
2003 8 39015 no prev year
2003 9 39015 no prev year
2003 10 39015 no prev year
2009 4 39015 no next year
2009 5 39015 no next year
2009 6 39015 no next year
2009 7 39015 no next year
2009 8 39015 no next year
2009 9 39015 no next year
2009 10 39015 no next year
2003 4 39017 no prev year
2003 5 39017 no prev year
2003 6 39017 no prev year
2003 7 39017 no prev year
2003 8 39017 no prev year
2003 9 39017 no prev year
2003 10 39017 no prev year
2009 4 39017 no next year
2009 5 39017 no next year
2009 6 39017 no next year
2009 7 39017 no next year
2009 8 39017 no next year
2009 9 39017 no next year
2009 10 39017 no next year
2003 4 39019 no prev year
2003 5 39019 no prev year
2003 6 39019 no prev year
2003 7 39019 no prev year
2003 8 39019 no prev year
2003 9 39019 no prev year
2003 10 39019 no prev year
2009 4 39019 no next year
2009 5 39019 no next year
2009 6 39019 no next year
2009 7 

,AGEGRP,FIPS,YEAR,TOT_FEMALE,TOT_FEMALE_next_year,TOT_FEMALE_prev_year
0,4,39001,2003,931,917,0
7,4,39001,2004,917,931,931
14,4,39001,2005,931,940,917
21,4,39001,2006,940,965,931
28,4,39001,2007,965,955,940
...,...,...,...,...,...,...
966,4,39039,2008,1289,1309,1323
973,4,39039,2009,1309,0,1289
980,4,39041,2003,4114,4273,0
987,4,39041,2004,4273,4412,4114


In [7]:
df.columns

Index(['SUMLEV', 'STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'YEAR', 'AGEGRP',
       'TOT_FEMALE', 'WA_FEMALE', 'BA_FEMALE', 'IA_FEMALE', 'AA_FEMALE',
       'NA_FEMALE', 'TOM_FEMALE', 'WAC_FEMALE', 'BAC_FEMALE', 'IAC_FEMALE',
       'AAC_FEMALE', 'NAC_FEMALE', 'NH_FEMALE', 'NHWA_FEMALE', 'NHBA_FEMALE',
       'NHIA_FEMALE', 'NHAA_FEMALE', 'NHNA_FEMALE', 'NHTOM_FEMALE',
       'NHWAC_FEMALE', 'NHBAC_FEMALE', 'NHIAC_FEMALE', 'NHAAC_FEMALE',
       'NHNAC_FEMALE', 'H_FEMALE', 'HWA_FEMALE', 'HBA_FEMALE', 'HIA_FEMALE',
       'HAA_FEMALE', 'HNA_FEMALE', 'HTOM_FEMALE', 'HWAC_FEMALE', 'HBAC_FEMALE',
       'HIAC_FEMALE', 'HAAC_FEMALE', 'HNAC_FEMALE', 'FIPS',
       'TOT_FEMALE_next_year', 'TOT_FEMALE_prev_year', 'MONTH'],
      dtype='object')

In [11]:
# expand each row, duplicating it 12 times, and adding in a new "Month" column taking value 1 - 12
# we can do this because currently, each row is unique in terms of year-fips-agegrp combination
df['MONTH'] = [[i for i in range(1,13)]] * len(df)
df = df.explode('MONTH')

# for 2003 years, months should only be 6 - 12; get rid of 2003 years months 1-5 
df = df[(df.YEAR != 2003) | ((df.YEAR == 2003) & (df.MONTH > 5))]

# now we can modify the values 
# month 6 = existing value [i.e. doesn't change]
# current year month 7-12
# new value = (previous_estimate + [next_estimate - previous_estimate]/12*i)
for col in ['TOT_FEMALE']:
    difference = df[col+ '_next_year'] - df[col]

    display(difference)
    k = df[[col]] + difference*(df[['MONTH']] - 6)
    print("hi")
    display(k)
    
    df.loc[df['MONTH'] > 6, col] = k[k['MONTH']>6]

# next year month 1-5 are modified 


# now for each of the following columns, we want to modify the value of months 7-12:
# 
# for col in ['tot_female', 'nhwa_female', 'nhba_female', 'nhia_female', 'nhaa_female', 'nhna_female', 'nhtom_fem', 'h_fem']:
# for col in ['TOT_FEMALE']:
#     df.loc[df['MONTH'] > 6, col] = df

# duplicate each row to get 12, add in a month column taking value 1 - 12
# for each row in [tot_female, nhwa_female, nhba_female, nhia_female, nhaa_female, nhna_female. nhtom_fem, h_fem]:
#      each value = population current year + 

# df.columns
# df[['AGEGRP', 'FIPS', 'YEAR', 'MONTH', 'TOT_FEMALE', 'TOT_FEMALE_next_year', 'TOT_FEMALE_prev_year']]

df[(df['AGEGRP'] == 4) & (df['YEAR'] == 2005)][['AGEGRP', 'FIPS', 'YEAR', 'MONTH', 'TOT_FEMALE', 'TOT_FEMALE_next_year', 'TOT_FEMALE_prev_year']]

,TOT_FEMALE,TOT_FEMALE_next_year
0,NaN,NaN
0,NaN,NaN
0,NaN,NaN
0,NaN,NaN
0,NaN,NaN
...,...,...
999,NaN,NaN
999,NaN,NaN
999,NaN,NaN
999,NaN,NaN


hi


,MONTH,TOT_FEMALE,TOT_FEMALE_next_year
0,NaN,NaN,NaN
0,NaN,NaN,NaN
0,NaN,NaN,NaN
0,NaN,NaN,NaN
0,NaN,NaN,NaN
...,...,...,...
999,NaN,NaN,NaN
999,NaN,NaN,NaN
999,NaN,NaN,NaN
999,NaN,NaN,NaN


,AGEGRP,FIPS,YEAR,MONTH,TOT_FEMALE,TOT_FEMALE_next_year,TOT_FEMALE_prev_year
14,4,39001,2005,1,931.0,940,917
14,4,39001,2005,2,931.0,940,917
14,4,39001,2005,3,931.0,940,917
14,4,39001,2005,4,931.0,940,917
14,4,39001,2005,5,931.0,940,917
...,...,...,...,...,...,...,...
994,4,39041,2005,8,NaN,0,4273
994,4,39041,2005,9,NaN,0,4273
994,4,39041,2005,10,NaN,0,4273
994,4,39041,2005,11,NaN,0,4273


In [9]:
df[(df['AGEGRP'] == 4) & (df['YEAR'] == 2005)][['AGEGRP', 'FIPS', 'YEAR', 'MONTH', 'TOT_FEMALE', 'TOT_FEMALE_next_year', 'TOT_FEMALE_prev_year']]

,AGEGRP,FIPS,YEAR,MONTH,TOT_FEMALE,TOT_FEMALE_next_year,TOT_FEMALE_prev_year
14,4,39001,2005,1,931.0,940,917
14,4,39001,2005,2,931.0,940,917
14,4,39001,2005,3,931.0,940,917
14,4,39001,2005,4,931.0,940,917
14,4,39001,2005,5,931.0,940,917
...,...,...,...,...,...,...,...
994,4,39041,2005,8,NaN,0,4273
994,4,39041,2005,9,NaN,0,4273
994,4,39041,2005,10,NaN,0,4273
994,4,39041,2005,11,NaN,0,4273


In [53]:
df['YEAR'].unique()

array([2003, 2004, 2005, 2006, 2007, 2008, 2009])